In [23]:
import pandas as pd
from collections import Counter

In [24]:
train_data = pd.read_csv('./combined-swell-classification-eda-train-dataset.csv')
test_data = pd.read_csv('./combined-swell-classification-eda-test-dataset.csv')
val_data = pd.read_csv('./combined-swell-classification-eda-validation-dataset.csv')

train_X = train_data
test_X = test_data
val_X = val_data

# train_Y, test_Y, val_Y
train_Y = train_X["condition"]
test_Y = test_X["condition"]
val_Y = val_X["condition"]

In [25]:
# train_X
del train_X["NasaTLX class"]
del train_X["Condition Label"]
del train_X["NasaTLX Label"]
del train_X["condition"]

# test_X
del test_X["NasaTLX class"]
del test_X["Condition Label"]
del test_X["NasaTLX Label"]
del test_X["condition"]

# val_X
del val_X["NasaTLX class"]
del val_X["Condition Label"]
del val_X["NasaTLX Label"]
del val_X["condition"]

In [26]:
class Node:
    def __init__(self):
        self.split_feature = None
        self.split_point = None
        self.result = None
        self.childs = None
        

keep going!


In [76]:
class DecisionTree:
    
    def __init__(self):
        self.root = None
        
    def Gini_Impurity(self, data_Y):  # input label dataset of a group
        impurity = 1
        label_counts = Counter(data_Y)
        for label in label_counts:
            p_of_label = label_counts[label] / len(data_Y)
            impurity -= p_of_label ** 2
        return impurity
    
    def Information_Gain(self, unsplited_data_Y, splited_data_Y):
        gain = Gini_Impurity(unsplited_data_Y)
        # print(gain)
        # print(splited_data_Y)
        for subset in splited_data_Y:
            # print("-", Gini_Impurity(subset), " X ( ", len(subset), " / ", len(unsplited_data_Y), " )")
            gain -= Gini_Impurity(subset) * (len(subset)/ len(unsplited_data_Y))
        return gain
    
    def Split(self, data_X, data_Y, column):
        data_X_subsets = []    # 분할 후의 data_X 그룹을 저장하는 배열
        data_Y_subsets = []    # 분할 후의 data_Y 그룹을 저장하는 배열
        split_point = 0.0      # 최적 분할 기준 값
        split_point_gain = 0.0  # 최적 분할 지점에서의 Information Gain
        '''
        print("=-=- Before reset Index =-=-=-")
        print("**** data_X ****")
        print(data_X[column])
        print("**** data_Y ****")
        print(data_Y)
        '''
        data_X = data_X.sort_values(by=column)
        data_Y = data_Y[data_X.index]
        data_X = data_X.reset_index(drop=True)
        data_Y = data_Y.reset_index(drop=True)
        '''
        print("=-=- After reset Index =-=-=-")
        print(data_X.index)
        print(data_X[column])
        print(data_Y)

        print(data_X[column].iloc[5], data_Y[5])
        '''
        for i in range(1, len(data_Y)):
            candidate_splited_data_X = []
            candidate_splited_data_Y = []
            if data_Y[i-1] != data_Y[i]:
                # print(i, data_Y[i-1], data_Y[i])
                candidate_point = (data_X[column].iloc[i-1] + data_X[column].iloc[i]) / 2
                candidate_splited_data_Y.append(data_Y[:i])
                candidate_splited_data_Y.append(data_Y[i:])
                gain = Information_Gain(data_Y, candidate_splited_data_Y)
                if gain > split_point_gain:
                    candidate_splited_data_X.append(data_X[:i])
                    candidate_splited_data_X.append(data_X[i:])
                    split_point = candidate_point
                    split_point_gain = gain
                    data_X_subsets = candidate_splited_data_X
                    data_Y_subsets = candidate_splited_data_Y
                    # print("== Updated :: ", split_point_gain, split_point)
        return split_point_gain, split_point, data_X_subsets, data_Y_subsets
    
    def Find_Best_Split(self, data_X, data_Y):
        # print("=-=-New Group=-=-")
        best_feature = ''    # 데이터를 분할 할 feature
        best_gain = 0.0       # 데이터를 특정 feature로 분할했을 때 가장 높게 측정된 Information Gain
        best_split_point = 0.0
        for column in data_X.columns:
            # print("check column :: ", column)
            gain, split_point = Split(data_X, data_Y, column)[0:2]
            if gain > best_gain:
                best_gain = gain
                best_feature = column
                best_split_point = split_point
        return best_feature, best_gain, best_split_point
    
    def fit(self, data_X, data_Y):
        root = Node()

        data_X = data_X.reset_index(drop=True)
        data_Y = data_Y.reset_index(drop=True)

        best_feature, best_gain, best_split_point = Find_Best_Split(data_X, data_Y)
        if best_gain == 0:
            # print("== No Split")
            root.result = data_Y[0]
            # print(root.result)
            return root
        data_X_subsets, data_Y_subsets = Split(data_X, data_Y, best_feature)[2:]

        # print("== Split :: ", best_feature, best_gain)

        childs = []
        for i in range(len(data_X_subsets)):
            childs.append(fit(data_X_subsets[i], data_Y_subsets[i]))

        root.split_feature = best_feature
        root.split_point = best_split_point
        root.childs = childs
        self.root = root
        
    def predict

In [81]:
newDT = DecisionTree()
print(newDT)

newDT.fit(test_X.head(100), test_Y.head(100))
print(newDT.root)